# 第三题：手动实现K-means

实验内容：
1. 实现一个K-means聚类算法，并在Breast_Cancer_Wisconsin数据集上完成聚类任务
2. 计算外部指标FMI和NMI
3. 对聚类结果可视化

# 1. 导入模块

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# 2. 导入数据集

In [ ]:
data = pd.read_csv('data\Breast_Cancer_Wisconsin\data')

In [ ]:
data = data.values 
data_x = data[:,2:-1]
data_y = data[:,1:2]
print(data_x.shape)

# 3. 欧式距离的实现

In [ ]:
def compute_distance(X, y):
    '''
    计算样本矩阵X与类中心y之间的欧氏距离
    '''
    # YOUR CODE HERE
    X = X.astype(float)
    y = y.astype(float)
    distance = np.sqrt(np.sum(np.square(X - y), axis=1))
    return distance

In [ ]:
print(compute_distance(np.array([[0, 0], [0, 1]]), np.array([0, 1]))) # [ 1.  0.]
print(compute_distance(np.array([[0, 0], [0, 1]]), np.array([1, 1]))) # [ 1.41421356  1.        ]

下面开始实现K-means聚类算法

In [ ]:
class myKmeans:
    def __init__(self, n_clusters, max_iter = 100):
        '''
        初始化，三个成员变量
        
        '''
        self.n_clusters = n_clusters
        self.max_iter = int(max_iter)
        self.centroids = None
    
    def choose_centroid(self, X):
        '''
        选取类簇中心
        
        '''
        centroids = X[np.random.choice(np.arange(len(X)), self.n_clusters, replace = False), :]
        return centroids
    
    def compute_label(self, X):
        '''
        给定样本矩阵X，结合类中心矩阵self.centroids，计算样本矩阵X内每个样本属于哪个类簇
        
        '''
        distances = np.empty((len(X), self.n_clusters))
        for index in range(len(self.centroids)):
            # 计算样本矩阵X所有样本到当前类中心的距离，存储在distances中的第index列中
            # YOUR CODE HERE
            distances[:, index] = compute_distance(X, self.centroids[index])
        # print(distances)
        # 取distances每行最小值的下标，这个下标就是这个样本属于的类簇的标记
        # YOUR CODE HERE
        labels = np.argmin(distances, axis=1)
        return labels
    
    def fit(self, X):
        '''
        聚类，包含类中心初始化，类中心优化两个部分
        
        '''
        self.centroids = self.choose_centroid(X)
        for epoch in range(self.max_iter):
            labels = self.compute_label(X)
            for index in range(self.n_clusters):
                
                # 重新计算第 index 个类中心，对属于这个类簇的样本取均值
                # YOUR CODE HERE
                self.centroids[index, :] = np.mean(X[labels == index], axis=0)

模型训练

In [ ]:
## YOUR CODE HERE
# 初始化一个2类簇的模型
model = myKmeans(n_clusters=2)
model.fit(data_x)
y_hat = model.compute_label(data_x)

聚类结果统计

In [ ]:
def getResult(data_y,y_hat):
    true_labels = data_y.reshape(-1)
    cluster = {}
    # 构造簇
    for i in range(len(y_hat)):
        cluster_label = y_hat[i]
        if cluster_label not in cluster:
            cluster[cluster_label] = {}
    # 构造簇内类别标签
    for cluster_label in cluster:
        for true_label in list(set(true_labels)):
            cluster[cluster_label][true_label] = 0
    # 添加簇内数据
    for i in range(len(y_hat)):
            cluster_label = y_hat[i]
            cluster[cluster_label][true_labels[i]] +=1
    # 按照簇序号排序
    cluster = dict(sorted(cluster.items(),key = lambda x:x[0]))
    return cluster
cluster = getResult(data_y,y_hat)
print(cluster)

# 5. 聚类结果可视化

In [ ]:
def draw_bar(cluster_data):
    # 构造绘图数据
    y_data = {}
    for cluster_label in cluster_data:
        for true_label in cluster_data[cluster_label]:
            y_data[true_label] = []
        break
    for cluster_label in cluster_data:
        for true_label,num in cluster_data[cluster_label].items():
            y_data[true_label] +=[num]

    # 绘图
    bar_width = 0.35
    t = 0  # 偏移量
    for key,data in y_data.items():
        plt.bar(np.arange(len(data))+t,data,label = key,width = bar_width)
        t+=bar_width

    labels = ["cluster "+str(l) for l in cluster_data]
    plt.xticks(np.arange(len(data))+bar_width-0.2,labels)
    plt.title("Cluster result")
    plt.legend()
draw_bar(cluster)

# 6. 评价指标

这里，我们选用两个外部指标，FMI和NMI。

In [ ]:
from sklearn.metrics import normalized_mutual_info_score
from sklearn.metrics import fowlkes_mallows_score

In [ ]:
# YOUR CODE HERE
print("NMI for selected features: ", normalized_mutual_info_score(data_y.reshape(-1), model.compute_label(data_x)))
print("FMI for selected features: ", fowlkes_mallows_score(data_y.reshape(-1), model.compute_label(data_x)))